## Sterowanie robotem przy pomocy kontrolera - ipywidgets 
---
Głównym celem użycia biblioteki ipywidgets jest prostota połączenia kontrolera do robota.   
Zacznijmy od zaimportowania biblioteki.   
ipywidgets pozwoli nam stworzyć interaktywne widgety w środowisku Jupytier

### Podłączenie kontrolera

Pierwszą rzeczą, którą chcemy zrobić, jest podłączenie kontrolera do komputera.   
Następnie musimy określić `index` naszego pada - jeżeli masz podłączony tylko jeden pad  
spróbuj pominąć ten krok.

1. Odwiedź stronę [http://html5gamepad.com](http://html5gamepad.com),  
2. Naciśnij przyciski na używanym gamepadzie,
3. Zapamiętaj ``index`` gamepada.

Kolejnnym krokiem jest stworzenie instancji widgetu `Controller`, którego będziemy używać do sterowania naszym robotem.  
Widżet `Controller` przyjmuje parametr `index` z wcześniejszego kroku.

Na końcu wyświetlimy widget w konsoli

In [ ]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0)  # replace with index of your controller

display(controller)

Jeżeli pad się poprawnie połączył powinin wyświetlić się komunikat ``Connect gamepad and press any button``. Spróbuj ponaciskać przyciski!

### Połączenie robota

Wykorzystamy bibliotekę [traitlets](https://traitlets.readthedocs.io/en/stable/) ponieważ użyta jest już wcześniej plikach robota. Interfejsowanie się ze zmiennymi tej biblioteki będzie prostsze jeżeli jej użyjemy.  
  
### W paru punktach o traitlets
1. Biblioteka 'wrapuje' podstawowe zmienne (float, int, string) w dodatkowe funkcje,
2. zmienne definiuje się: `steering = traitlets.Float()`,
3. nowe funkcje to na przykład:   
    a) observe - pozwala na wywołanie określonej funkcji wraz z każdą modyfikacją zmiennej   
    b) dlink - pozwala na "połączenie" dwóch zmiennych - ich wartości zawszę będą sobie odpowiadać   
   
Polecam dalszą samodzielną analizę biblioteki [traitlets](https://traitlets.readthedocs.io/en/stable/)



**UWAGA po paru sekundach po uruchomienu następnej instrukcji robot będzie sterowany padem**


In [ ]:
import traitlets
from jetracer.nvidia_racecar import NvidiaRacecar
car = NvidiaRacecar()

def update_steer(change):
    car.steering = change['new']
    
def update_throttle(change):
    car.throttle = change['new']*-1    

controller.axes[0].observe(update_steer, names='value')
controller.axes[3].observe(update_throttle, names='value')

Jeżeli wszystko zadziałało poprawnie robot powinien być sterowany padem!

### Pora na obraz

Przy pomocy biblioteki `ipwidgets` możemy również wyświtlać do termianlu grafiki. Świetnym testem tej funkcji będzie przesyłanie obrazu z kamery robota.   
Zacznijmy od stworzenia obiektu klasy `Image`


In [ ]:
img = widgets.Image(format='jpeg', width=640, height=480)

W tym momencie nasz obiekt `img` jest pusty. Musimy go zapełnić obrazem z kamery, aby to zrobić zaimportujmy pliki Jetsona   
   
Następnie tworzymy obiekt kamery i włączmy go

In [ ]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=640, height=480)
camera.running = True

Przed połączeniem zmiennych musimy dokonąć pewnego preprocesingu obrazu.   
Nasza klasa kamery obecnie generuje wartości tylko w formacie BGR8 (niebieski, zielony, czerwony, 8-bitowy),  
podczas gdy nasz widget obrazu akceptuje wartości w skompresowanym formacie *JPEG*. Użyjemy do tego funkcji `bgr8_to_jpeg` z plików Jetsona `jetcam.utils`   
   
Ostatnim krokiem jest połączenie dwóch zmiennych przy pomocy `traitlets`

In [ ]:
from jetcam.utils import bgr8_to_jpeg

camera_link = traitlets.dlink((camera, 'value'), (img, 'value'), transform=bgr8_to_jpeg)
display(img)

Kamera powinna działać!

Możesz kliknąć prawym przyciskiem myszy na obraz i wybrać ``Create New View for Output`` aby obraz pojawił się w nowym oknie
  
Aby użyć kamery w innym notbooku zresetuj lub wyłącz kernel.
  
### To wszystko

Miłej zabawy    
stworzone przez: Michał Kozłowski